In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow())
print(f"{current_date}")

2025-03-05 07:49:35.877410


/tmp/ipykernel_186114/2672588006.py:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_date = pd.to_datetime(datetime.utcnow())


In [3]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [4]:
from datetime import datetime, timedelta, timezone
current_date = pd.to_datetime(datetime.now(timezone.utc)).floor("h")

In [5]:
current_date.to_datetime64()

numpy.datetime64('2025-03-05T07:00:00.000000000')

In [6]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=29)

In [7]:
fetch_data_to

Timestamp('2025-03-05 07:00:00+0000', tz='UTC')

In [8]:
fetch_data_from

Timestamp('2025-02-04 07:00:00+0000', tz='UTC')

In [9]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_taxi_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: A DataFrame containing the simulated production data.
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Ensure input timestamps are timezone-aware (UTC)
    from_date = pd.to_datetime(from_date, utc=True)
    to_date = pd.to_datetime(to_date, utc=True)

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)

    # Load and filter data for the historical period
    rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
    
    # Ensure pickup_datetime is timezone-aware
    rides_from['pickup_datetime'] = pd.to_datetime(rides_from['pickup_datetime'], utc=True)
    
    # Filter using tz-aware timestamps
    rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        
        # Ensure pickup_datetime is timezone-aware
        rides_to['pickup_datetime'] = pd.to_datetime(rides_to['pickup_datetime'], utc=True)
        
        # Filter using tz-aware timestamps
        rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]

        # Combine the filtered data
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from

    # Shift the data forward by 52 weeks to simulate recent data
    rides['pickup_datetime'] += timedelta(weeks=52)

    # Ensure sorting works by keeping all timestamps consistent
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides


In [10]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

Successfully downloaded data for 2024-02.
Loading data for 2024-02...
Total records: 3,007,526
Valid records: 2,954,709
Records dropped: 52,817 (1.76%)
Successfully processed data for 2024-02.
Combining all monthly data...
Data loading and processing complete!
Successfully downloaded data for 2024-03.
Loading data for 2024-03...
Total records: 3,582,628
Valid records: 3,518,066
Records dropped: 64,562 (1.80%)
Successfully processed data for 2024-03.
Combining all monthly data...
Data loading and processing complete!


In [11]:
rides

,pickup_datetime,pickup_location_id
807560,2025-02-12 16:25:44+00:00,2
1114261,2025-02-15 16:56:40+00:00,2
67330,2025-02-04 18:17:57+00:00,3
95845,2025-02-05 05:47:46+00:00,3
124636,2025-02-05 12:33:19+00:00,3
...,...,...
2947934,2025-03-05 06:56:35+00:00,263
2947682,2025-03-05 06:57:37+00:00,263
2948949,2025-03-05 06:58:42+00:00,263
2947809,2025-03-05 06:59:48+00:00,263


In [12]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [13]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-02-04 07:00:00+00:00,2,0
1,2025-02-04 08:00:00+00:00,2,0
2,2025-02-04 09:00:00+00:00,2,0
3,2025-02-04 10:00:00+00:00,2,0
4,2025-02-04 11:00:00+00:00,2,0
...,...,...,...
175387,2025-03-05 02:00:00+00:00,263,5
175388,2025-03-05 03:00:00+00:00,263,3
175389,2025-03-05 04:00:00+00:00,263,4
175390,2025-03-05 05:00:00+00:00,263,19


In [14]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175392 entries, 0 to 175391
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype              
---  ------              --------------   -----              
 0   pickup_hour         175392 non-null  datetime64[ns, UTC]
 1   pickup_location_id  175392 non-null  int16              
 2   rides               175392 non-null  int16              
dtypes: datetime64[ns, UTC](1), int16(2)
memory usage: 2.0 MB


In [15]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)



/home/saideshith1905/venv3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-03-05 02:50:08,506 INFO: Initializing external client
2025-03-05 02:50:08,507 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 02:50:09,242 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215689


In [16]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |████████████████████████| Rows 175392/175392 | Elapsed Time: 00:24 | Remaining Time: 00:00
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/saideshi/Resources/jobs/time_series_hourly_feature_group_1_offline_fg_materialization/config_1741135197373) to trigger the materialization job again.


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)